In [1]:
import rasterio
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
from torchvision.models.segmentation import (
    lraspp_mobilenet_v3_large,
    deeplabv3_resnet50,
    deeplabv3_mobilenet_v3_large,
)
from torchvision.models import mobilenetv3, ResNet50_Weights

In [3]:
model = deeplabv3_mobilenet_v3_large(
    weights_backbone="IMAGENET1K_V1",
    num_classes=1,
)

In [6]:
trans1 = mobilenetv3.MobileNet_V3_Large_Weights.IMAGENET1K_V1.transforms()
trans2 = ResNet50_Weights.IMAGENET1K_V1.transforms()

In [7]:
trans1, trans2

(ImageClassification(
     crop_size=[224]
     resize_size=[256]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BILINEAR
 ),
 ImageClassification(
     crop_size=[224]
     resize_size=[256]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BILINEAR
 ))

In [2]:

INPUT_DIR = "supervised_data"
DATASET_DIR = "segtrain_data"
input_label_mapping_train = {
    f"{INPUT_DIR}/PNW lakes/PNW_20230612_3_4.tif": f"{INPUT_DIR}/PNW lakes/PNW_20230612_3_4_label.tif",
    f'{INPUT_DIR}/Willamette/Willamette_20230606_06.tif': f'{INPUT_DIR}/Willamette/Willamette_20230606_06_label.tif',
    f'{INPUT_DIR}/Willamette/Willamette_20230606_07.tif': f'{INPUT_DIR}/Willamette/Willamette_20230606_07_label.tif',
}
input_label_mapping_val = {
    f"{INPUT_DIR}/PNW lakes/PNW_20230612_1_1.tif": f"{INPUT_DIR}/PNW lakes/PNW_20230612_1_1_label.tif",
}
input_label_mapping_test = {
    f'{INPUT_DIR}/Willamette/Willamette_20230606_08.tif': f'{INPUT_DIR}/Willamette/Willamette_20230606_08_label.tif',
}

In [57]:

output_chunks = []
input_chunks = []
for k,v in input_label_mapping_test.items():
    input_dataset = rasterio.open(k)
    input_data = input_dataset.read()
    output_dataset = rasterio.open(v)
    output_data = output_dataset.read()

    _, s1,s2 = output_data.shape
    chunk_size = 256
    for i in range(s1//chunk_size):
        for j in range(s2//chunk_size):
            input_chunk = input_data[:, i*chunk_size : (i+1)*chunk_size, j*chunk_size : (j+1)*chunk_size,]
            output_chunk = output_data[:, i*chunk_size : (i+1)*chunk_size, j*chunk_size : (j+1)*chunk_size,]
            input_chunks.append(input_chunk)
            output_chunks.append(output_chunk)
    #         break
    #     break
    # break
num_pos_samples = 0
ctr = 0
for inp, outp in zip(input_chunks, output_chunks):
    out_path = os.path.join(DATASET_DIR, "test", f"{ctr:06d}.npy")
    with open(out_path, "wb") as f:
        np.savez(
            f,
            input=inp,
            output=outp,
        )
    ctr += 1
    if np.sum(outp)>0:
        num_pos_samples += 1
print(num_pos_samples)

290


In [58]:
inp.shape, outp.shape

((4, 256, 256), (1, 256, 256))

In [54]:
873/17000

0.05135294117647059